<a href="https://colab.research.google.com/github/ykitaguchi77/statistics_for_articles/blob/main/Surgically_induced_astigmatism_eyelid_malignant_tumor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Surgically induced astigmatism and higher order abberation after surgery of the eyelid malignant tumor**

In [41]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd

xlsx_path = "/content/drive/MyDrive/発表/日本眼形成再建外科学会/202306 HOA after wide excision/眼瞼悪性腫瘍波面_2.xlsx"
excel_data = pd.read_excel(xlsx_path)

df = excel_data.drop(['ID', 'Name'], axis=1) #idとnameを削除

# Display the first few rows of the dataframe
df.head()


In [43]:
from datetime import datetime

#年齢の算出
def calculate_age_robust(birth_date, surgery_date):
    try:
        # Handling potential inconsistencies in date formats
        birth_str = str(birth_date).split("/")[0]  # Handling dates with slashes
        surgery_str = str(surgery_date).split("/")[0]  # Handling dates with slashes

        birth = datetime.strptime(birth_str, "%Y%m%d")
        surgery = datetime.strptime(surgery_str, "%Y%m%d")
        age = surgery.year - birth.year - ((surgery.month, surgery.day) < (birth.month, birth.day))
        return age
    except Exception as e:
        # Returning a placeholder in case of any errors
        return None

# Applying the updated function to the dataframe
df['age'] = df.apply(lambda x: calculate_age_robust(x['生年月日'], x['手術日']), axis=1)

# Calculating statistical values for the Age_at_Surgery column
age_statistics = df['age'].describe()

print(age_statistics)

count    21.000000
mean     70.000000
std      13.270268
min      38.000000
25%      66.000000
50%      73.000000
75%      78.000000
max      87.000000
Name: age, dtype: float64


In [45]:
#その他の統計量
# Extracting relevant columns: Gender, Left/Right (Unnamed: 4), and Defect size
gender_lr_defect = df[['性別', 'side', 'Defect']]

# Renaming columns for clarity
gender_lr_defect.columns = ['Gender', 'Left/Right', 'Defect Size']

# Calculating statistics for Defect Size
defect_stats = gender_lr_defect['Defect Size'].describe()

# Counting occurrences for Gender and Left/Right
gender_counts = gender_lr_defect['Gender'].value_counts()
left_right_counts = gender_lr_defect['Left/Right'].value_counts()

print(gender_counts)
print("")
print(left_right_counts)
print("")
print(defect_stats)




F    15
M     6
Name: Gender, dtype: int64

R    11
L    10
Name: Left/Right, dtype: int64

count    21.000000
mean     16.142857
std       7.220012
min       8.000000
25%      12.000000
50%      14.000000
75%      20.000000
max      34.000000
Name: Defect Size, dtype: float64


In [46]:
#上眼瞼と下眼瞼に分けて解析

# Splitting the dataframe based on the 'upper_lower' column values
df_lower = df[df["lower-upper"] == 0]
df_upper = df[df["lower-upper"] == 1]

# Displaying the first few rows of each new dataframe to verify
print(f"上眼瞼: {len(df_upper)}例、下眼瞼: {len(df_lower)}例")
#df_lower.head(), df_upper.head()


上眼瞼: 7例、下眼瞼: 14例


In [93]:
from scipy.stats import mannwhitneyu

# 上眼瞼と下眼瞼の比較 --> 有意差なし
# Function to perform Mann-Whitney U test and return the p-value
def perform_mannwhitneyu_test(column):
    stat, p_value = mannwhitneyu(df_lower[column].dropna(), df_upper[column].dropna())
    return p_value

# Performing the test for each column
columns_to_compare = ["SIA_D", "Δasg_0", "Δasg_45", "ΔHOA"]
p_values = {column: perform_mannwhitneyu_test(column) for column in columns_to_compare}

p_values

{'SIA_D': 0.3223082215342278,
 'Δasg_0': 0.6888200894392846,
 'Δasg_45': 0.360061919504644,
 'ΔHOA': 0.5352597179222567}

In [48]:
#SIA_D, Δasg_0, Δasg_45に関する統計量

# Splitting the dataframe again based on the 'lower-upper' column values
df_all = df
df_lower = df[df["lower-upper"] == 0]
df_upper = df[df["lower-upper"] == 1]

# Comparing the columns 'SIA_D', 'Δasg_0', 'Δasg_45' in df_upper and df_lower

# Selecting the relevant columns for comparison
columns_to_compare = ['SIA_D', 'Δasg_0', 'Δasg_45']

# Calculating statistical values for these columns in both dataframes
stats_df_all = df_all[columns_to_compare].describe()
stats_df_lower = df_lower[columns_to_compare].describe()
stats_df_upper = df_upper[columns_to_compare].describe()

print(stats_df_all)
print("")
print(stats_df_lower)
print("")
print(stats_df_upper)


           SIA_D     Δasg_0    Δasg_45
count  21.000000  21.000000  21.000000
mean    0.884828   0.140827   0.061317
std     0.569896   0.861801   0.615933
min     0.200000  -1.560450  -1.318164
25%     0.552330  -0.301631  -0.240736
50%     0.731663  -0.010470   0.205561
75%     1.063482   0.630383   0.431334
max     2.426709   2.101558   0.882085

           SIA_D     Δasg_0    Δasg_45
count  14.000000  14.000000  14.000000
mean    0.741406   0.058397   0.184310
std     0.367172   0.655101   0.507314
min     0.200000  -0.972569  -0.736869
25%     0.517273  -0.286174  -0.202471
50%     0.671924  -0.013673   0.323809
75%     1.009928   0.447196   0.467991
max     1.375730   1.348246   0.882085

          SIA_D    Δasg_0   Δasg_45
count  7.000000  7.000000  7.000000
mean   1.171672  0.305687 -0.184668
std    0.804062  1.224037  0.775281
min    0.304378 -1.560450 -1.318164
25%    0.595550 -0.456905 -0.632861
50%    0.873601  0.550848  0.040425
75%    1.702957  0.980832  0.303812
max    2

In [60]:
from scipy.stats import ttest_ind, f_oneway
import statsmodels.api as sm
from scipy.stats import spearmanr


# Extracting the relevant columns for SIA_D analysis
sia_d_data = df[['性別', 'side', 'Defect', 'SIA_D']]
sia_d_data.columns = ['Gender', 'Left/Right', 'Defect Size', 'SIA_D']

# Removing rows with missing SIA_D values
sia_d_data = sia_d_data.dropna(subset=['SIA_D'])

# Univariate Analysis
# T-tests for Gender and Left/Right
mannwhitneyu_gender = mannwhitneyu(sia_d_data[sia_d_data['Gender'] == 'M']['SIA_D'],
                         sia_d_data[sia_d_data['Gender'] == 'F']['SIA_D'])
mannwhitneyu_left_right = mannwhitneyu(sia_d_data[sia_d_data['Left/Right'] == 'L']['SIA_D'],
                             sia_d_data[sia_d_data['Left/Right'] == 'R']['SIA_D'])

# Calculating Spearman correlation and p-value between age and SIA_D
spearman_age_result = spearmanr(df['age'], df['SIA_D'])
# Spearman correlation result and p-value
spearman_age_corr_value, spearman_age_p_value = spearman_age_result


# Calculating Spearman correlation and p-value between Defect and SIA_D
spearman_defect_result = spearmanr(df['Defect'], df['SIA_D'])
# Spearman correlation result and p-value
spearman_defect_corr_value, spearman_defect_p_value = spearman_defect_result


print(f"Gender: {mannwhitneyu_gender}")
print(f"Side: {mannwhitneyu_left_right}")
print(f"Age: r={spearman_age_corr_value}, p={spearman_age_p_value}")
print(f"Defect: r={spearman_defect_corr_value}, p={spearman_defect_p_value}")



Gender: MannwhitneyuResult(statistic=63.0, pvalue=0.17805543269939555)
Side: MannwhitneyuResult(statistic=43.0, pvalue=0.4180519085356006)
Age: r=0.17844427004489263, p=0.4389871964685451
Defect: r=-0.005859394868315979, p=0.9798898360235521


In [91]:
from scipy.stats import wilcoxon

#preと3Mの高次収差の比較

# Performing Wilcoxon Signed-Rank Test for HOA_pre and HOA_3M
wilcoxon_test_result = wilcoxon(df['HOA_pre'], df['HOA_3M'])

# Wilcoxon test result
wilcoxon_test_statistic, wilcoxon_test_pvalue = wilcoxon_test_result
print(wilcoxon_test_statistic, wilcoxon_test_pvalue)


# Calculating descriptive statistics for HOA_pre and HOA_3M
hoa_pre_stats = df['HOA_pre'].describe()
hoa_3m_stats = df['HOA_3M'].describe()

print(hoa_pre_stats)
print(hoa_3m_stats)



35.0 0.0037527084350585938
count    21.000000
mean      0.180619
std       0.077137
min       0.082000
25%       0.110000
50%       0.179000
75%       0.214000
max       0.353000
Name: HOA_pre, dtype: float64
count    21.000000
mean      0.264667
std       0.171358
min       0.097000
25%       0.153000
50%       0.221000
75%       0.296000
max       0.798000
Name: HOA_3M, dtype: float64


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Plotting a scatter plot of HOA_pre and HOA_3M with equal axis scales
plt.figure(figsize=(8, 8))
sns.scatterplot(data=df, x='HOA_pre', y='HOA_3M')
plt.title('Scatter Plot of HOA_pre vs HOA_3M')
plt.xlabel('HOA_pre (Pre-Surgery)')
plt.ylabel('HOA_3M (3 Months Post-Surgery)')
plt.grid(True)

# Setting equal scales for both axes
plt.xlim(df[['HOA_pre', 'HOA_3M']].min().min(), df[['HOA_pre', 'HOA_3M']].max().max())
plt.ylim(df[['HOA_pre', 'HOA_3M']].min().min(), df[['HOA_pre', 'HOA_3M']].max().max())

plt.show()


from scipy.stats import pearsonr

# Calculating Pearson correlation and p-value between HOA_pre and HOA_3M
pearson_corr_hoa_result = pearsonr(df['HOA_pre'], df['HOA_3M'])

# Pearson correlation result and p-value for HOA_pre and HOA_3M
pearson_corr_hoa_value, pearson_p_value_hoa = pearson_corr_hoa_result
print(f"r={pearson_corr_hoa_value}, p={pearson_p_value_hoa}")





# Plotting a scatter plot with a regression line
plt.figure(figsize=(8, 8))
sns.regplot(data=df, x='HOA_pre', y='HOA_3M', ci=None)  # ci=None removes the confidence interval
plt.title('Scatter Plot of HOA_pre vs HOA_3M with Regression Line')
plt.xlabel('HOA_pre (Pre-Surgery)')
plt.ylabel('HOA_3M (3 Months Post-Surgery)')
plt.grid(True)

# Setting equal scales for both axes
plt.xlim(df[['HOA_pre', 'HOA_3M']].min().min(), df[['HOA_pre', 'HOA_3M']].max().max())
plt.ylim(df[['HOA_pre', 'HOA_3M']].min().min(), df[['HOA_pre', 'HOA_3M']].max().max())

plt.show()



###術前後の乱視比較

In [73]:
#術前後の乱視の比較
# Performing Wilcoxon Signed-Rank Test for CASIA_CYL_pre and CASIA_CYL_3M
wilcoxon_test_casia_cyl = wilcoxon(df['CASIA_CYL_pre'], df['CASIA_CYL_3M'])

# Wilcoxon test result for CASIA_CYL_pre and CASIA_CYL_3M
wilcoxon_test_casia_cyl_statistic, wilcoxon_test_casia_cyl_pvalue = wilcoxon_test_casia_cyl
print(wilcoxon_test_casia_cyl_statistic, wilcoxon_test_casia_cyl_pvalue)

# Calculating descriptive statistics for CASIA_CYL_pre and CASIA_CYL_3M
casia_cyl_pre_stats = df['CASIA_CYL_pre'].describe()
casia_cyl_3m_stats = df['CASIA_CYL_3M'].describe()

print(casia_cyl_pre_stats)
print(casia_cyl_3m_stats)



104.5 0.7334699630737305
count    21.000000
mean     -1.069524
std       0.681839
min      -2.400000
25%      -1.400000
50%      -0.900000
75%      -0.700000
max      -0.200000
Name: CASIA_CYL_pre, dtype: float64
count    21.000000
mean     -1.185714
std       0.699490
min      -3.100000
25%      -1.500000
50%      -1.100000
75%      -0.600000
max      -0.200000
Name: CASIA_CYL_3M, dtype: float64


In [ ]:
# Plotting a scatter plot of CASIA_CYL_pre and CASIA_CYL_3M
plt.figure(figsize=(8, 8))
sns.scatterplot(data=df, x='CASIA_CYL_pre', y='CASIA_CYL_3M')
plt.title('Scatter Plot of CASIA_CYL_pre vs CASIA_CYL_3M')
plt.xlabel('CASIA_CYL_pre (Pre-Surgery)')
plt.ylabel('CASIA_CYL_3M (3 Months Post-Surgery)')
plt.grid(True)

# Setting equal scales for both axes
plt.xlim(df[['CASIA_CYL_pre', 'CASIA_CYL_3M']].min().min(), df[['CASIA_CYL_pre', 'CASIA_CYL_3M']].max().max())
plt.ylim(df[['CASIA_CYL_pre', 'CASIA_CYL_3M']].min().min(), df[['CASIA_CYL_pre', 'CASIA_CYL_3M']].max().max())

plt.show()


from scipy.stats import pearsonr

# Calculating Pearson correlation and p-value between CASIA_CYL_pre and CASIA_CYL_3M
pearson_corr_casia_cyl_result = pearsonr(df['CASIA_CYL_pre'], df['CASIA_CYL_3M'])

# Pearson correlation result and p-value
pearson_corr_casia_cyl_value, pearson_p_value_casia_cyl = pearson_corr_casia_cyl_result
print(f"r={pearson_corr_casia_cyl_value}, p={pearson_p_value_casia_cyl}")


# Plotting a scatter plot with a regression line
plt.figure(figsize=(8, 8))
sns.regplot(data=df, x='CASIA_CYL_pre', y='CASIA_CYL_3M', ci=None)  # ci=None removes the confidence interval
plt.title('Scatter Plot of CASIA_CYL_pre vs CASIA_CYL_3M with Regression Line')
plt.xlabel('CASIA_CYL_pre (Pre-Surgery)')
plt.ylabel('CASIA_CYL_3M (3 Months Post-Surgery)')
plt.grid(True)

# Setting equal scales for both axes
plt.xlim(df[['CASIA_CYL_pre', 'CASIA_CYL_3M']].min().min(), df[['CASIA_CYL_pre', 'CASIA_CYL_3M']].max().max())
plt.ylim(df[['CASIA_CYL_pre', 'CASIA_CYL_3M']].min().min(), df[['CASIA_CYL_pre', 'CASIA_CYL_3M']].max().max())

plt.show()

In [88]:
# Performing Wilcoxon Signed-Rank Test for asg_0_pre and asg_0_3M
wilcoxon_test_asg_0 = wilcoxon(df['asg_0_pre'], df['asg_0_3M'])

# Wilcoxon test result for asg_0_pre and asg_0_3M
wilcoxon_test_asg_0_statistic, wilcoxon_test_asg_0_pvalue = wilcoxon_test_asg_0
wilcoxon_test_asg_0_statistic, wilcoxon_test_asg_0_pvalue

(101.0, 0.6332969665527344)

In [89]:
# Performing Wilcoxon Signed-Rank Test for asg_0_pre and asg_0_3M
wilcoxon_test_asg_0 = wilcoxon(df['asg_45_pre'], df['asg_45_3M'])

# Wilcoxon test result for asg_0_pre and asg_0_3M
wilcoxon_test_asg_0_statistic, wilcoxon_test_asg_0_pvalue = wilcoxon_test_asg_0
wilcoxon_test_asg_0_statistic, wilcoxon_test_asg_0_pvalue

(89.0, 0.37372493743896484)